In [9]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
from numba import vectorize

In [3]:
# looking at numpy's implicit multithreading
n = 50
m = 1000
for i in range(n):
    X = np.random.randn(m, m)
    λ = np.linalg.eigvals(X)

In [15]:
# using numba's jit compilation + numpy's implicit multithreading together
@vectorize('float64(float64,float64)',target='parallel')
def f_vec(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 5000)
x, y = np.meshgrid(grid, grid)

np.max(f_vec(x, y))  # run once to compile

np.float64(0.9999992797121728)

In [20]:
# multithreaded loops in numba
# base approach

""" 
w_t -> wealth at time t
R_t -> gross rate of return on assets at time t
s -> savings rate of the household
y_t -> labor income at time t
Both R and y are modelled as independent draws ffrom a lognormal distribution
"""
from numpy.random import randn
from numba import njit

@njit
def h(w, r=0.1, s=0.3, v1=0.1, v2=1.0):
    """
    Updates household wealth.
    """

    # Draw shocks
    R = np.exp(v1 * randn()) * (1 + r)
    y = np.exp(v2 * randn())

    # Update wealth
    w = R * s * w + y
    return w

In [21]:
# sim 50k households for 1k periods to get a long run median
@njit
def compute_long_run_median(w0=1, T=1000, num_reps=50_000):

    obs = np.empty(num_reps)
    for i in range(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        obs[i] = w

    return np.median(obs)

In [22]:
%%time
compute_long_run_median()

CPU times: total: 3.12 s
Wall time: 3.22 s


1.8444705123154324

In [23]:
# the numba version using prange
from numba import prange

@njit(parallel=True)
def compute_long_run_median_parallel(w0=1, T=1000, num_reps=50_000):

    obs = np.empty(num_reps)
    for i in prange(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        obs[i] = w

    return np.median(obs)

In [24]:
%%time
compute_long_run_median_parallel()

CPU times: total: 4.42 s
Wall time: 928 ms


1.8266961452503567

# Ex 18.1
Parallelize the Monte-Carlo estimator for $\pi$.

In [ ]:
from numba import njit, prange
import numpy as np
import time

n = 10**8

@njit
def calc_pi(n):  # n = number of iterations, higher n = more accurate
    p = np.random.rand(n)
    q = np.random.rand(n)

    quadrant = 0  # points in the quadrant (d<1)
    square = 0  # points in the square and not in the quadrant (d>1)

    distance = np.sqrt((p ** 2) + (q ** 2))

    for d in prange(len(distance)):
        if distance[d] <= 1:
            quadrant += 1
        else:
            square += 1

    ratio = quadrant / (quadrant + square)
    π = ratio * 4
    return π

# time the numba implementation
start_numba = time.time()
pi_estimate = calc_pi(n)
end_numba = time.time()
print(pi_estimate)

print(f"Parallelized implementation: {end_numba-start_numba:.6f} seconds")
# for ref, numba with njit was 1.629 secs and with prange its about 1.823 for n = 10**8
# the tasks are so small that parallelizing isnt giving any sizeable benefits

3.14178696
Parallelized implementation: 1.823604 seconds
